In [1]:
import torch 
print(torch.__version__)

2.0.0+cu118


In [2]:
import os 
import tqdm
import imageio
import numpy as np
import time
import random
import math
import scipy.io as sio
from PIL import Image#

In [3]:
fx = 365.481
fy = 365.481
cx = 257.346
cy = 210.347
 

SAMPLE_NUM = 2048 # 2048
fps_sample_num=24 # 512
K = 20  # max frame limit for temporal rank

save_path = '4DV/train_T/T'

try:
    os.makedirs(save_path)
except OSError:
    pass

def main():
    data_path = 'train_data_pointnpy'  #'train_data_pointnpy'
    sub_Files = os.listdir(data_path)
    sub_Files.sort()
    print(sub_Files)

    for s_fileName in sub_Files[1:]:#确认Drik文件           #lx#Traverse 17 settings folders
        print(s_fileName)
        videoPath = os.path.join(data_path, s_fileName)
        #print(videoPath)
        if os.path.isdir(videoPath):#lx#Determine whether it is a directory
                #print(s_fileName)
                video_Files = os.listdir(videoPath)
                video_Files.sort()
                #print(video_Files)
                video_index=0
                video_num=len(video_Files[1:])
                #print(video_num)
                
                for video_FileName in video_Files[1:]:#lx#Traverse the sample folder
                    #print(video_FileName)
                    video_index=video_index+1

                    filename = video_FileName +'.npy'
                    file = os.path.join(save_path, filename)
                    #print(file)
                    if os.path.isfile(file):
                        continue
                    
                    pointPath = os.path.join(videoPath,video_FileName)
                    imgNames = os.listdir(pointPath)
                    imgNames.sort()
                    print(len(imgNames))
                  
                    #确定合并帧
                    m=len(imgNames)
                    s=int(m/fps_sample_num)
                    n=list(range(1,m,s))
                    l_now=len(n)
                    dele=0
                    while l_now>24:
                        if(dele==0):
                            n=n[1:]
                            l_now=l_now-1
                            dele=1
                        else:
                            n=n[:-1]
                            l_now=l_now-1
                            dele=0
                    
                    #合并帧
                    temp = np.zeros([24,2048,3])
                    index=0
                    for i in n:
                        #print(imgNames[i])
                        each=imgNames[i]
                        real_path= os.path.join(videoPath,video_FileName,each)
                        #print(real_path)
                        real_data = np.load(real_path, allow_pickle=True)  # 类型是numpy array
                        for j in range(2048):
                            #print(index)
                            temp[index,j,:]=real_data[2*j,:]
                            j=j+1
                        #temp[index,:,:]=real_data
                        index=index+1
                        
                    #print(temp.shape)
                    
                    #对数组归一化
                    max_x = temp[:,:,0].max()
                    max_y = temp[:,:,1].max()
                    max_z = temp[:,:,2].max()
                    min_x = temp[:,:,0].min()
                    min_y = temp[:,:,1].min()
                    min_z = temp[:,:,2].min()
                
                    x_len = max_x - min_x
                    y_len = max_y - min_y              
                    z_len = max_z - min_z
                
                    x_center = (max_x + min_x)/2
                    y_center = (max_y + min_y)/2
                    z_center = (max_z + min_z)/2
                
                    temp[:,:,0]=(temp[:,:,0]-x_center)/y_len
                    temp[:,:,1]=(temp[:,:,1]-y_center)/y_len
                    temp[:,:,2]=(temp[:,:,2]-z_center)/y_len
                    #print(temp[1,:,:])
                    np.save(file,temp)

In [ ]:
main()